In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# 不使用 GPU
# 必須放在import後的第一格
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
# 限制記憶體
# 必須放在import後的第一格
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:    
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

In [4]:
img_width, img_height = 224, 224
train_data_dir = './data/train'
#train_data_dir = os.path.join(os.getcwd(), 'test')           
validation_data_dir = './data/train'
nb_train_samples = 1000
nb_validation_samples = 1000
epochs = 5
batch_size = 200
input_shape = (img_height, img_width, 3)
num_classes = 2 # 類別個數

In [5]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
#Create your own input format (here 224x224x3)
input = Input(shape=(img_width, img_height,3),name = 'image_input')

In [7]:
#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

In [8]:
#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
#x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(256, activation='relu', name='fc2')(x)
x = Dense(num_classes, activation='softmax', name='predictions')(x)
#x = Dense(2, activation='sigmoid')(x)

In [9]:
#Create your own model 
model = Model(inputs=input, outputs=x)

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(learning_rate=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [11]:
train_datagen = ImageDataGenerator(
    # rescale=1. / 255,
    rotation_range = 0.2, # 旋轉
    shear_range=0.2, # 裁剪
    zoom_range=0.2,  # 放大/縮小
    horizontal_flip=False) # 水平翻轉

In [12]:
SAVE_PATH="./data/train1"
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False, # 不放回抽樣
    # color_mode="grayscale", # no of channel = 1
    # save_to_dir=SAVE_PATH, # save to file
    class_mode='categorical')

Found 45 images belonging to 2 classes.


In [13]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False, # 不放回抽樣
    # color_mode="grayscale", # no of channel = 1
    #save_to_dir=SAVE_PATH,
    class_mode='categorical')

Found 45 images belonging to 2 classes.


In [14]:
train_history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
5/5 [==============================] - 20s 1s/step - loss: 8.7672 - accuracy: 0.4667 - val_loss: 2.4404 - val_accuracy: 0.6667


In [15]:
img_path = './data/train/cats/cat.0.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x1 = image.img_to_array(img)
x1 = np.expand_dims(x1, axis=0)
x1 = preprocess_input(x1)

labels=['cat', 'dog']
predictions = model.predict(x1)
print('predict class:', labels[np.argmax(predictions)])
print('predict:', predictions.ravel())


predict class: cat
predict: [1.0000000e+00 2.0255838e-08]
